In [1]:
import seaborn as sns
import csv
import pandas as pd
import os
import openpyxl
import matplotlib.pyplot as plt
import datetime


In [7]:
def get_eval(file_path, file_name) -> pd.DataFrame:
    filepath = '{}/{}'.format(file_path, file_name)
    
    with open(filepath,'r') as f:

        reader = csv.reader(f)
        l = [row for row in reader]
        windows = l[0]
        strides = l[1]
        df = pd.DataFrame(l[2:], index = windows, columns = strides)
        return df

In [10]:
subseq_len = 8
filepath = 'scratch/eval'
filename = 'annoy_{}.csv'.format(subseq_len)
df = get_eval(filepath, filename)

In [16]:
sns.lineplot()

Index(['100', '120'], dtype='object')


In [ ]:
#sketch_dim, kmer_len, window, stride, fpr, foundr, vectorizing_time build_time, query_time
def get_df(filepath, filename):
    filepath = '{}/{}'.format(filepath, filename)
    
    df = pd.read_csv(filepath,names=['sketch_dim','kmer','window','stride','fpr','missr','vectorizing_time','build_time','query_time'])
   
    return df.sort_values(by=['sketch_dim','kmer','window','stride']).reset_index(drop=True)

In [ ]:
def df_to_excel(df, file_path, excel_name, sheet_name, header ='', row = 0, col = 0):
        filepath = '{}/{}.xlsx'.format(file_path, excel_name)
        first_time = False
        
        if not os.path.isfile(filepath):
            df_empty = pd.DataFrame()
            df_empty.to_excel(filepath)
            first_time = True
            
        workbook = openpyxl.load_workbook(filepath)
        
        if first_time and not (sheet_name in workbook.sheetnames):
            ex_sheet = workbook['Sheet1']
            ex_sheet.title = sheet_name
            workbook.save(filepath)
        workbook.close()
        
        with pd.ExcelWriter(filepath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer: 
            df.to_excel(writer, sheet_name=sheet_name, startrow = row, startcol = col)
            workbook = writer.book
            worksheet = workbook[sheet_name]
            worksheet.cell(row=row+1, column = col+1).value = header
            workbook.save(filepath)
            workbook.close()

In [ ]:
def write_to_excel(filepath, excel_name, df):
    def format_time(num):
        tmp = str(datetime.timedelta(seconds=round(num,2))).split('.')
        print(num)
        return tmp[0]+ '.' + tmp[1][:2]

    df_group = df.groupby(['sketch_dim','kmer'])
    
    offset=0
    check_key = -1
    for key, table in df_group:
        sketch_dim = key[0]
        if check_key != sketch_dim:
            offset = 0
        kmer = key[1]
        #table['vectorizing_time'] = table['vectorizing_time'].apply(lambda num: str(datetime.timedelta(seconds=round(num,2))))
        table['build_time'] = table['build_time'].apply(lambda num: str(datetime.timedelta(seconds=round(num,2))))
        table['query_time'] = table['query_time'].apply(lambda num: str(datetime.timedelta(seconds=round(num,2))))
        df_table = table.pivot(index='window',columns='stride',values='fpr')
        #df_vec = table.pivot(index='window',columns='stride',values='vectorizing_time')
        df_build = table.pivot(index='window',columns='stride',values='build_time')
        df_query = table.pivot(index='window',columns='stride',values='query_time')
        row_num, col_num = df_table.shape
        df_to_excel(df_table, filepath, excel_name=excel_name,sheet_name=str(sketch_dim), header='t={}'.format(kmer),row=offset)  
        #df_to_excel(df_vec, filepath, excel_name=excel_name,sheet_name=str(sketch_dim), header='vec_time'.format(kmer),row=offset,col = col_num + 1) 
        df_to_excel(df_build, filepath, excel_name=excel_name,sheet_name=str(sketch_dim), header='build_time'.format(kmer),row=offset, col = 2*(col_num + 1)) 
        df_to_excel(df_query, filepath, excel_name=excel_name,sheet_name=str(sketch_dim), header='query_time'.format(kmer),row=offset, col = 3*(col_num + 1)) 
        offset+=row_num
        check_key = sketch_dim

In [ ]:
k_search = ''
index_type = 'annoy'
vectorizer = 'tensor_sketch'
filepath = 'eval'
filename = '{}_{}_{}nns.csv'.format(index_type, vectorizer,k_search)
df_all = get_df(filepath, filename)

In [ ]:
window = 40
df_selec = df_all.loc[(df_all['window'] == window)].reset_index()
print(df_selec)
sns.lineplot(data=df_selec,x='stride',y='vectorizing_time',hue='kmer',linewidth=2.0).set_title('vec_time for 256')

In [ ]:
sns.lineplot(data=df_selec,x='stride',y='build_time',hue='kmer',linewidth=2.0).set_title('build_time for 256')

In [ ]:
sns.lineplot(data=df_selec,x='stride',y='query_time',hue='kmer',linewidth=2.0).set_title('query_time for 256')